
<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ</b></h3>

<h3 style="text-align: center;"><b>Итоговый проект. Базовый поток. Весна 2021</b></h3>
<h3 style="text-align: center;"><b>Тема проекта: "Discover the mysteries of the Maya"</b></h3>
<p style="align: center;"><img align=center src="https://biasvariancelabs.github.io/maya_challenge/images/remotesensing-12-02215-g003.jpg" width=600 height=400/></p>





Выполнил: Иван Кочкуров

Stepik User ID: 327138733

https://github.com/matroskinsk

Данная работа выполнена на основе baseline предоставленного Григорием Лелейтнером

#Данные


In [1]:
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#%cd /content/drive/MyDrive/MayaChallenge/

# Эта библиотека нужна для чтения спутниковых снимков -- у них особый формат файла.
!easy_install GDAL



Mounted at /content/drive
Searching for GDAL
Best match: GDAL 2.2.2
Adding GDAL 2.2.2 to easy-install.pth file

Using /usr/lib/python2.7/dist-packages
Processing dependencies for GDAL
Finished processing dependencies for GDAL


In [ ]:
!unzip -u '/content/drive/MyDrive/data/DiscoverMayaChallenge_data.zip' -d '/content/drive/MyDrive/data/DiscoverMayaChallenge_data/

/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 1: syntax error: unexpected end of file


##Определение путей к папкам

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import os

from osgeo import gdal, gdal_array

from pathlib import Path
from copy import copy
from torch.utils.data import Dataset, random_split
import torch
from torch import nn
from torchvision import transforms as T
from torch.nn import functional as F
#from segmentation_transforms import Compose, RandomHorizontalFlip, RandomCrop, Normalize, RandomResize
import torchvision.transforms.functional as transforms_F

import pickle

from pathlib import Path

#from chactun_dataset import ChactunDataset, UpsampleSentinelToLidar

mask_train_path = Path('/content/drive/MyDrive/train/train_masks')
lidar_train_path = Path('/content/drive/MyDrive/train/lidar_train')
sent1_train_path = Path('/content/drive/MyDrive/data/Sentinel1_train')

##ChactunDataset(Dataset)

In [4]:
from pathlib import Path
from copy import copy
from torch.utils.data import Dataset, random_split
import torch
from torch import nn
from torchvision import transforms as T
from torch.nn import functional as F
import torchvision.transforms.functional as transforms_F
import os
from PIL import Image
import numpy as np
from osgeo import gdal, gdal_array
import fastai
import PIL


default_sentinel1_bands =  [
        97, #mean vv
        99, #std vv
        103, #mean vh
        105 #std vh
    ]
default_sentinel2_bands = [0] 



def gather_tile_ids(path):
    ids = []

    files = os.listdir(path)
    for name in files:
        if 'tile_' not in name:
            continue
        ids.append(int(name.split('_')[1]))
    return set(ids)


class ChactunDataset(Dataset):
    classes = ['aguada', 'building', 'platform']

    def __init__(self, 
                root_folder, 
                is_train=True, 
                transform=None,
                sentinel1_bands=None,
                sentinel2_bands=None):
        super().__init__()
        if sentinel1_bands is None:
            sentinel1_bands = default_sentinel1_bands
        if sentinel2_bands is None:
            sentinel2_bands = default_sentinel2_bands
        self.sentinel1_bands = sentinel1_bands
        self.sentinel2_bands = sentinel2_bands

        self.root = Path(root_folder)
        self.type = 'train' if is_train else 'test'
        self.transform = transform
        self.num_bands = 3 + len(sentinel1_bands + sentinel2_bands)

        if is_train:
            self.masks_path = self.root / f'{self.type}_masks'
        self.lidar_path = self.root / f'lidar_{self.type}'
        self.sentinel1_path = self.root / f'Sentinel1_{self.type}'
        self.sentinel2_path = self.root / f'Sentinel2_{self.type}'

        self.ids = list(self.gather_complete_tile_ids())

    def gather_complete_tile_ids(self):
        if self.type == 'train':
            masks_ids = gather_tile_ids(self.masks_path)
        lidar_ids = gather_tile_ids(self.lidar_path)
        sentinel1_ids = gather_tile_ids(self.sentinel1_path)
        sentinel2_ids = gather_tile_ids(self.sentinel2_path)

        complete_ids = lidar_ids & sentinel1_ids & sentinel2_ids
        all_ids = lidar_ids | sentinel1_ids | sentinel2_ids
        if self.type == 'train':
            complete_ids &= masks_ids
            all_ids |= masks_ids

        if len(lidar_ids) < len(all_ids):
            print(f'Lidar images are missing ids: [{all_ids - lidar_ids}]')
        if len(sentinel1_ids) < len(all_ids):
            print(f'Sentinel1 images are missing ids: [{all_ids - sentinel1_ids}]')
        if len(sentinel2_ids) < len(all_ids):
            print(f'Sentinel2 images are missing ids: [{all_ids - sentinel2_ids}]')
        
        return complete_ids

    def load_lidar(self, id):
        pil_img = Image.open(self.lidar_path / f'tile_{id}_lidar.tif')
        return T.functional.to_tensor(pil_img)

    def load_sentinel1(self, id):
        if len(self.sentinel1_bands) == 0:
            return None
            
        img = gdal_array.LoadFile(str(self.sentinel1_path / f'tile_{id}_S1.tiff'))
        return torch.from_numpy(img)[self.sentinel1_bands]

    def load_sentinel2(self, id):
        if len(self.sentinel2_bands) == 0:
            return None

        img = gdal_array.LoadFile(str(self.sentinel2_path / f'tile_{id}_S2.tiff'))
        return torch.from_numpy(img)[self.sentinel2_bands]

    def load_masks(self, id):
        if self.type == 'test':
            raise NotImplementedError('This dataset is meant for testing and doesn\'t have masks.')
        pil_masks = [np.array(Image.open(self.masks_path / f'tile_{id}_mask_{mask_class}.tif'))
                        for mask_class in self.classes]
        tensor_masks = torch.from_numpy(np.stack(pil_masks)).type(torch.float32) / 255
        # invert masks. Originally 1 = not present, 0 = present
        tensor_masks = 1 - tensor_masks
        return tensor_masks

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, i):
        id = self.ids[i]

        x = {'lidar': self.load_lidar(id), 
             'sentinel1': self.load_sentinel1(id), 
             'sentinel2': self.load_sentinel2(id)}
        if self.type == 'train':
            y = self.load_masks(id)
        else:
            y = None

        if self.transform:
            x, y = self.transform(x, y)
        return x, y


class UpsampleSentinelToLidar(object):
    def __call__(self, images, target=None):
        lidar_size = images['lidar'].shape[1:]

        if images['sentinel1'] is not None:
            images['sentinel1'] = T.functional.resize(images['sentinel1'], lidar_size,
                                                  interpolation=PIL.Image.BILINEAR)
        if images['sentinel2'] is not None:
            images['sentinel2'] = T.functional.resize(images['sentinel2'], lidar_size,
                                                  interpolation=PIL.Image.BILINEAR)
        
        merged_image = torch.cat([img_channels for img_channels in images.values() 
                                    if img_channels is not None], dim=0)
        return merged_image, target

##Transform

In [9]:
import numpy as np
from PIL import Image
import random

import torch
from torchvision import transforms as T
from torchvision.transforms import functional as F

def pad_if_smaller(img, size, fill=0):
    min_size = min(img.shape[1:])
    if min_size < size:
        ow, oh = img.size
        padh = size - oh if oh < size else 0
        padw = size - ow if ow < size else 0
        img = F.pad(img, (0, 0, padw, padh), fill=fill)
    return img


class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target


class RandomResize(object):
    def __init__(self, min_size, max_size=None):
        self.min_size = min_size
        if max_size is None:
            max_size = min_size
        self.max_size = max_size

    def __call__(self, image, target):
        size = random.randint(self.min_size, self.max_size)
        image = F.resize(image, size)
        if target is not None:
            target = F.resize(target, size, interpolation=Image.NEAREST)
        return image, target


class RandomHorizontalFlip(object):
    def __init__(self, flip_prob):
        self.flip_prob = flip_prob

    def __call__(self, image, target):
        if random.random() < self.flip_prob:
            image = F.hflip(image)
            if target is not None:
                target = F.hflip(target)
        return image, target


class RandomCrop(object):
    def __init__(self, size):
        self.size = size

    def __call__(self, image, target):
        image = pad_if_smaller(image, self.size)
        target = pad_if_smaller(target, self.size, fill=255)
        crop_params = T.RandomCrop.get_params(image, (self.size, self.size))
        image = F.crop(image, *crop_params)
        if target is not None:
            target = F.crop(target, *crop_params)
        return image, target


class CenterCrop(object):
    def __init__(self, size):
        self.size = size

    def __call__(self, image, target):
        image = F.center_crop(image, self.size)
        if target is not None:
            target = F.center_crop(target, self.size)
        return image, target


class ToTensor(object):
    def __call__(self, image, target):
        image = F.to_tensor(image)
        if target is not None:
            target = torch.as_tensor(np.array(target), dtype=torch.int64)
        return image, target


class Normalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, image, target):
        image = F.normalize(image, mean=self.mean, std=self.std)
        return image, target

##DoDict

Инженерная хитрость. Когда вы запускаете много экспериментов удобно весь конфиг эксперимента класть в один объект. В данном случае это DotDict, это словарь, к элементам которого можно получать доступ и через точку. Потом нам это ее пригодится для wandb - еще одной штуки, которая сильно облегчает контроль за экспериментами.

In [5]:
class DotDict(dict):
    def __getattr__(self, attr):
        return self.get(attr)

    def __setattr__(self, key, value):
        self.__setitem__(key, value)

config = DotDict()
# with open('./dataset_stats.pickle', 'rb+') as f:
#     config.mean, config.std = pickle.load(f)
# Здесь мы задаем конечный размер картинки, которую выдает нам датасет и которая подается в модель.
# Если вы делаете предсказания не для соревнования, то поставьте здесь 250, тогда размер выходных масок сегментации будет
# 250х250, как и картинка, которая подавалась на вход. 
# Для соревнования размер 480х480
config.resize_min =480
config.resize_max = 480
config.crop_size = 480
config.pretrained = True
config.num_classes = 3#len(ChactunDataset.classes)
config.batch_size = 4
config.epochs = 27
config.lr = 1e-4
config.momentum = 0.9
config.num_workers = 2#multiprocessing.cpu_count()
#экспериметы с добавлением каналов
config.sentinel1_bands = [97, 99, 103, 105, 109, 111, 115, 117]#ascending descending- VV HV mean, std[97, 99, 103, 105]#
config.sentinel2_bands = [216, 217]#NIR#[210, 211, 212]# sorted([2 + i for i in range(0, 221, 13)] + [3 + i for i in range(0, 221, 13)] + [4 + i for i in range(0, 221, 13)])#RGB
config.in_chanel = (len(config.sentinel1_bands) + len(config.sentinel2_bands) + 3)
config.pin_memory=True

Рассмотрим формат наших данных. Есть две части датасета с ответами и без. К части с ответами принадлежат папки "lidar_train", "Sentinel1_train", "Sentinel2_train", "train_masks".  К части без ответов "lidar_test", "Sentinel1_test", "Sentinel2_test". 

Файлы в lidar_train и lidar_test  имеют расширение tif. Они имеют 3 канала и могут быть открыты с помощью библиотеки PIL. Как и любая другая картинка.

Файлы в "Senintel1_..." и "Sentinel2_..." имеют расширение .tiff. В таких файлах могут храниться картинки с большим чем 3 числом каналов. Их нельзя открыть с поомщью PIL. Поэтому мы применяем библиотеку osgeo. В ней есть специальный метод который сразу загружает .tiff картинку как numpy array (игнорируя метаданные, например о дате снимка итд). Этот метод называется gdal_array. 

При загрузке картинок ажно следить в каком формате они нам отдаются [C, H, W] (как принято в PyTorch) или [H,W,C] (как принято в numpy), а также в каком промежутке значения от 0 до 1 или от 0 до 255. От этого зависит как мы превратим картинку в torch тензор. Так, PIL картинку с лидара мы отправим в torchvision.transform.ToTensor, а картинки из gdal_array уже в нужном формате и мы их просто превратим в тензоры с помощью torch.from_numpy. 

Маски в "train_masks" назодятся в .tif файлах с одним каналом и их значения от 0 до 255. В коде загрузки мы руками приводим их к картинкам от 0 до 1 типа float32 с нужным порядком каналов ([C, H, W]).

Подробнее узнать, что означает каждый из каналов спутниковых снимков можно по ссылке https://biasvariancelabs.github.io/maya_challenge/res/S1%20and%20S2%20TIFF%20file%20structure.pdf. 

##GetDataset

In [10]:
# Здесь мы импортируем написанный нами класс для работы с датасетом. Комментарии по работе в самом py файле

def get_transofrms(h_flip_prob, resize_min, resize_max, crop_size, mean, std):
    train_transform = Compose([        
        UpsampleSentinelToLidar(),
        RandomHorizontalFlip(90),
        #RandomResize(300, 480),
        #RandomCrop(crop_size)
        #Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    test_transform = Compose([
        UpsampleSentinelToLidar()
    ])

    return train_transform, test_transform

def get_dataset(config, root='./data', val_size=0.2495):
    train_transform, test_transform = get_transofrms(
        config.h_flip_prob,
        config.resize_min,
        config.resize_max,
        config.crop_size,
        config.mean,
        config.std
    )

    # У нас есть два датасета, один с ответами, а другой без.
    # Первый датасет мы поделим на train часть и валидационную часть.
    ds = ChactunDataset(root, is_train=True, transform=train_transform,
                        sentinel1_bands=config.sentinel1_bands,
                        sentinel2_bands=config.sentinel2_bands)
    train_ds, val_ds = random_split(ds, [len(ds) - int(len(ds) * val_size), 
                                        int(len(ds) * val_size)])

    test_ds = ChactunDataset('/content/drive/MyDrive/test', is_train=False, transform=test_transform,
                             sentinel1_bands=config.sentinel1_bands,
                             sentinel2_bands=config.sentinel2_bands)

    return train_ds, val_ds, test_ds

In [11]:
from torch.utils.data import DataLoader

train_ds, val_ds, test_ds = get_dataset(config, '/content/drive/MyDrive/train')

train_dl = DataLoader(train_ds, batch_size=config.batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=config.batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=config.batch_size)
print(len(train_dl), len(val_dl), len(test_dl))

331 110 83


# Модель.

Мы будем использовать модель deeplabv3 с предобученными весами. Возьмем ее из стандартного репозитория моделей, которые уже есть в PyTorch. Выход этой стандартной модели - словарь с основным выходом и с дополнительным. Доп выход использовался при основном обучениия модели, чтобы бороться с затухающими градиентами. Сейчас он нам не понадобится. 

В модели мы заменим последний слой на слой с нужным нам количеством каналов на выходе. У нас есть три класса, поэтому мы хотим, чтобы модель отдавала нам тензор размерности [bathc_size, 3, original_h, original_w]. Поэтому заменяем последний слой на слой с тремя фильтрами 1 на 1. 

При этом наши маски сегментации независимы: каждый пиксель может одновременно принадлежать нескольким классам. Предобученная модель отдает нам результат применения последнего сверточного слоя без активации. Так как каждый пиксель может иметь несколько классов, то мы применяем не SoftMax, а сигмоиду + лосс BCELoss. https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html. 

**Интересный факт**: BCELoss принимает тензор с уже примененной сигмоидой, в то ввремя как для софтмакса существует лосс, который применяет его уже внутри себя, а на вход требует обычных logit'ов. Это связано с тем, что для софтмакса если применять его одновременно с лоссом можно использовать более стабильный расчет градиентов с меньшими численными ошибками.

##Save_model, Load_model

##Loss_func

In [ ]:
def focal_loss(y_real, y_pred, eps = 1e-8, gamma = 2):
  y_pred = y_pred.sigmoid() + eps
  y_real = y_real.view(-1) 
  y_pred = y_pred.view(-1) 
  
  your_loss =  -((1-y_pred)**gamma*y_real*y_pred.log()+(1-y_real)*(1-y_pred).log()) # hint: torch.clamp
  return your_loss.mean()

##Загрузка модели

In [12]:
from torchvision import models
in_chanel = config.in_chanel
# Здесь определяем нашу модель. По сути мы просто берем выход предобученной модели
# и применяем к нему сигмоиду. Подробнее узнать про интерфейс модели можно на торчхабе
# https://pytorch.org/hub/pytorch_vision_deeplabv3_resnet101/
# В данном случае она возвращает словарь с ключами out и aux.
class DeepLabV3(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.model = models.segmentation.deeplabv3_resnet101(
            pretrained=self.config.pretrained, progress=True
        )
# change first layer to work with different number of classes
        
        self.model.backbone.conv1 = nn.Conv2d(in_chanel , 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)#меняем колличество входных каналов
        self.model.classifier[4] = nn.Conv2d(256, self.config.num_classes, 1)

    def forward(self, X):
        return torch.sigmoid(self.model(X)['out'])

model = DeepLabV3(config)

Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth


In [ ]:
from torchvision import models
class DeepLabV3(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.model = models.segmentation.deeplabv3_resnet101(
            pretrained=self.config.pretrained, progress=True
        )

        self.model.classifier[4] = nn.Conv2d(256, self.config.num_classes, 1)

    def forward(self, x):
        return self.model(x)
model = DeepLabV3(config)

Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth


##Загрузка параметров модели

In [13]:
from torch import optim

# Стандартное определение лосса, оптимизатора и lr scheduler.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
crit =nn.BCELoss()#(reduction="mean")#nn.MSELoss(reduction='mean')#nn.CrossEntropyLoss#nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=config.momentum)#optim.Adam(model.parameters())#, lr=config.lr, eps=1e-08, weight_decay=0.05, amsgrad=False)#optim.Adam([dict(params=model.parameters(), lr=config.learning_rate), ])#optim.AdamW(model.parameters(), lr=0.001, eps=1e-08, weight_decay=0.05, amsgrad=False)#optim.SGD(model.parameters(), lr=config.lr, momentum=config.momentum)#optim.Adam(model.parameters())#, lr=config.lr, eps=1e-08, weight_decay=0.05, amsgrad=False)#optim.SGD(model.parameters(), lr=config.lr, momentum=config.momentum)#optim.Adam(model.parameters())# optim.AdamW(model.parameters(), lr=0.001, eps=1e-08, weight_decay=0.05, amsgrad=False)
sched = optim.lr_scheduler.OneCycleLR(optimizer, config.lr, 
                                      epochs=config.epochs, 
                                      steps_per_epoch=len(train_dl))
model = model.to(device)

##Metric

In [14]:
def discretize_segmentation_maps(probs, thresh):
    if thresh is None:
        thresh = [0.5, 0.5, 0.5]
    if isinstance(thresh, int):
        thresh = [thresh] * 3
    thresh = torch.from_numpy(np.array(thresh)).to(probs.device)
    return probs > thresh[:, None, None]

def get_ious(y_pred, y_true, thresh=None, eps=1e-7):# 
    y_pred = discretize_segmentation_maps(y_pred, thresh).float()
    y_true = y_true.float()
    with torch.no_grad():
        intersection = torch.sum(y_true * y_pred, dim=[2, 3])
        union = torch.sum(y_true, dim=[2, 3]) + torch.sum(y_pred, dim=[2, 3]) - intersection
        ious = ((intersection + eps) / (union + eps)).mean(dim=0)
    return ious

Помимио всего прочего я бы хотел, чтобы вы научились пользоваться wandb или его аналогом. Это инструмент, который позволяет во время обучения легко логировать значения метрик, градиентов в вашей модели + сохранять веса. Все эти данные уходят в облако, где уже строятся красивые графики. 


Чтобы понять как пользоваться wandb достаточно посмотреть одну страничку документации https://docs.wandb.ai/guides/integrations/pytorch. После зарегистрироваться и, когда выполнится код внизу и вас попросят авторизоваться, то перейти по ссылке и ввести свой код (прямо как с маунтом гугл диска). После этого в консоли появится еще больше текста и одна из ссылок будет вести на старницу с метриками. 

Особенно этот инструмент удобен, когда ваши расчеты проходят не в ноутбуке, а в .py файлах, где нет возможности отображать графики удобно.

#Wandb

Ввиду того что в основном обучение проводилось в основном на рабочей машине https://wandb.ai/ был заблокирован ввиду внутренних ограничений, в принципе через VPN он работал, но случались периодически отвалы что роняли полностью обучение, пришлось от него отказаться хотя штука конечно интересная и удобная.
В замен прикрутил простенькое логирование из которого можно достать данные для отчетов по обучению

In [ ]:
!pip install wandb

import wandb
wandb.init(config=config)
wandb.watch(model, log_freq=100)

ERROR: Operation cancelled by user


ModuleNotFoundError: ignored

#Обучение

##Старт обучения

In [15]:
from tqdm.notebook import tqdm, trange
log_loss = []
log_iou_class= []
for i in trange(config.epochs):
    model.train()
    c = 0
    print(f'EPOCH: {i + 1}')
    for X, y in tqdm(train_dl):
        c += 1
        X = X.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        pred = model(X)
        loss = crit(pred, y)
        loss.backward()
        optimizer.step()
        sched.step()
        
        #LOG metrics to wandb
        ious = get_ious(pred, y, config.prediction_thresh).cpu()
        metrics = {'iou_' + class_name: iou_score.item()
                for class_name, iou_score in zip(ChactunDataset.classes, ious)}
        {'avg_iou': ious.mean(), 'Loss': loss.item()}        
        log_loss.append({'avg_iou': ious.mean(), 'Loss': loss.item()} )  
        log_iou_class.append(metrics)
        if c % 50 == 0:          
            print({'avg_iou': ious.mean(), 'Loss': loss.item()}) 
            print(metrics)        
        #metrics['avg_iou'] = ious.mean()
        #metrics['loss'] = loss.item()
        #metrics['lr'] = sched.get_last_lr()
        #wandb.log(metrics)
        
    model.eval()
    losses = []
    ious = []
    for X, y in val_dl:
        X = X.to(device)
        y = y.to(device)

        with torch.no_grad():
            pred = model(X)
            losses.append(crit(pred, y).item())
            iou = get_ious(pred, y, config.prediction_thresh)
            ious.append(iou.cpu().numpy())
    metrics = {'val_iou_' + class_name: iou_score.item()
                for class_name, iou_score in zip(ChactunDataset.classes, np.stack(ious).mean(axis=0))}
    #sched.step(np.mean(losses))
    print(f'val_avg_iou: {np.array(ious).mean(axis=0).mean()}, val_loss: {np.mean(losses)}' )
    print(metrics)
    #metrics['val_avg_iou'] = np.array(ious).mean(axis=0).mean()
    #metrics['val_loss'] = np.mean(losses)
   # wandb.log(metrics)

EPOCH: 1


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:387: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


{'avg_iou': tensor(0.0028), 'Loss': 0.6634944677352905}
{'iou_aguada': 2.6047146210739225e-12, 'iou_building': 3.826410976265349e-12, 'iou_platform': 0.008311486802995205}
{'avg_iou': tensor(0.0868), 'Loss': 0.6553052663803101}
{'iou_aguada': 9.199112825653e-13, 'iou_building': 1.6514144652451934e-12, 'iou_platform': 0.260532408952713}
{'avg_iou': tensor(0.0069), 'Loss': 0.6505698561668396}
{'iou_aguada': 1.1730373616902767e-12, 'iou_building': 0.001366499694995582, 'iou_platform': 0.019355548545718193}
{'avg_iou': tensor(0.0844), 'Loss': 0.6423925161361694}
{'iou_aguada': 1.163178711335866e-12, 'iou_building': 0.000487807032186538, 'iou_platform': 0.25286200642585754}
{'avg_iou': tensor(0.0008), 'Loss': 0.6334540843963623}
{'iou_aguada': 1.2071083065404165e-12, 'iou_building': 0.0024718225467950106, 'iou_platform': 1.0910206071912398e-09}
{'avg_iou': tensor(0.0946), 'Loss': 0.6272597908973694}
{'iou_aguada': 5.1823935767747464e-12, 'iou_building': 0.02231835573911667, 'iou_platform': 

{'avg_iou': tensor(0.0011), 'Loss': 0.6135554313659668}
{'iou_aguada': 1.84123679844439e-12, 'iou_building': 0.0008588441996835172, 'iou_platform': 0.0025009200908243656}
{'avg_iou': tensor(0.0857), 'Loss': 0.5996025204658508}
{'iou_aguada': 2.8137665808447565e-12, 'iou_building': 0.007001624442636967, 'iou_platform': 0.25}
{'avg_iou': tensor(0.1104), 'Loss': 0.5880873203277588}
{'iou_aguada': 7.77360294190732e-12, 'iou_building': 0.01119640190154314, 'iou_platform': 0.320089191198349}
{'avg_iou': tensor(7.4627e-05), 'Loss': 0.5814719200134277}
{'iou_aguada': 2.35007326099268e-11, 'iou_building': 0.00022388060460798442, 'iou_platform': 3.1917465198594286e-11}
{'avg_iou': tensor(0.0881), 'Loss': 0.5729492902755737}
{'iou_aguada': 1.527265987288917e-11, 'iou_building': 0.014311367645859718, 'iou_platform': 0.25}
{'avg_iou': tensor(0.1714), 'Loss': 0.5572137236595154}
{'iou_aguada': 3.4829535089375696e-11, 'iou_building': 1.4500356644575607e-11, 'iou_platform': 0.5142865777015686}

val_av

{'avg_iou': tensor(0.0035), 'Loss': 0.5277755260467529}
{'iou_aguada': 2.5032149963966077e-11, 'iou_building': 0.0006637477781623602, 'iou_platform': 0.009974172338843346}
{'avg_iou': tensor(0.0833), 'Loss': 0.5114312767982483}
{'iou_aguada': 4.334672898398395e-11, 'iou_building': 0.25, 'iou_platform': 2.7187407880546743e-10}
{'avg_iou': tensor(0.0833), 'Loss': 0.5002633333206177}
{'iou_aguada': 0.25, 'iou_building': 2.3934031839756287e-11, 'iou_platform': 2.2018791304145857e-11}
{'avg_iou': tensor(0.6667), 'Loss': 0.4750996530056}
{'iou_aguada': 0.5, 'iou_building': 0.75, 'iou_platform': 0.75}
{'avg_iou': tensor(0.2500), 'Loss': 0.46563196182250977}
{'iou_aguada': 0.25, 'iou_building': 0.25, 'iou_platform': 0.25}
{'avg_iou': tensor(0.5000), 'Loss': 0.4436134099960327}
{'iou_aguada': 0.75, 'iou_building': 0.25, 'iou_platform': 0.5}

val_avg_iou: 0.5651714205741882, val_loss: 0.4310511816631664
{'val_iou_aguada': 0.8795454502105713, 'val_iou_building': 0.35915064811706543, 'val_iou_plat

...

{'avg_iou': tensor(0.5000), 'Loss': 0.06460829824209213}
{'iou_aguada': 1.0, 'iou_building': 0.25, 'iou_platform': 0.25}
{'avg_iou': tensor(0.3333), 'Loss': 0.11292627453804016}
{'iou_aguada': 1.0, 'iou_building': 2.563691793577849e-11, 'iou_platform': 8.57509029061898e-12}
{'avg_iou': tensor(0.3333), 'Loss': 0.08555250614881516}
{'iou_aguada': 1.0, 'iou_building': 4.650339141543114e-11, 'iou_platform': 1.9731780445675362e-11}
{'avg_iou': tensor(0.5833), 'Loss': 0.06658699363470078}
{'iou_aguada': 1.0, 'iou_building': 0.25, 'iou_platform': 0.5}
{'avg_iou': tensor(0.5000), 'Loss': 0.0688929557800293}
{'iou_aguada': 1.0, 'iou_building': 0.25, 'iou_platform': 0.25}
{'avg_iou': tensor(0.5000), 'Loss': 0.10247190296649933}
{'iou_aguada': 1.0, 'iou_building': 0.25, 'iou_platform': 0.25}

val_avg_iou: 0.6007575392723083, val_loss: 0.07188594900071621
{'val_iou_aguada': 0.9636363387107849, 'val_iou_building': 0.36818182468414307, 'val_iou_platform': 0.47045454382896423}
EPOCH: 24


{'avg_iou': tensor(0.6667), 'Loss': 0.05486350134015083}
{'iou_aguada': 1.0, 'iou_building': 0.5, 'iou_platform': 0.5}
{'avg_iou': tensor(0.5833), 'Loss': 0.06483595073223114}
{'iou_aguada': 1.0, 'iou_building': 0.5, 'iou_platform': 0.25}
{'avg_iou': tensor(0.8333), 'Loss': 0.05872275307774544}
{'iou_aguada': 1.0, 'iou_building': 0.75, 'iou_platform': 0.75}
{'avg_iou': tensor(0.5833), 'Loss': 0.09430183470249176}
{'iou_aguada': 0.75, 'iou_building': 0.5, 'iou_platform': 0.5}
{'avg_iou': tensor(0.6667), 'Loss': 0.07569459825754166}
{'iou_aguada': 1.0, 'iou_building': 0.25, 'iou_platform': 0.75}
{'avg_iou': tensor(0.4167), 'Loss': 0.07288814336061478}
{'iou_aguada': 1.0, 'iou_building': 4.9497107801332874e-11, 'iou_platform': 0.25}

val_avg_iou: 0.6007575392723083, val_loss: 0.07020944587209008
{'val_iou_aguada': 0.9636363387107849, 'val_iou_building': 0.36818182468414307, 'val_iou_platform': 0.47045454382896423}
EPOCH: 25


{'avg_iou': tensor(0.5833), 'Loss': 0.07298285514116287}
{'iou_aguada': 1.0, 'iou_building': 0.25, 'iou_platform': 0.5}
{'avg_iou': tensor(0.8333), 'Loss': 0.05176321417093277}
{'iou_aguada': 1.0, 'iou_building': 0.75, 'iou_platform': 0.75}
{'avg_iou': tensor(0.6667), 'Loss': 0.05811387673020363}
{'iou_aguada': 1.0, 'iou_building': 0.5, 'iou_platform': 0.5}
{'avg_iou': tensor(0.5833), 'Loss': 0.08216168731451035}
{'iou_aguada': 1.0, 'iou_building': 0.25, 'iou_platform': 0.5}
{'avg_iou': tensor(0.5000), 'Loss': 0.07926017791032791}
{'iou_aguada': 1.0, 'iou_building': 0.25, 'iou_platform': 0.25}
{'avg_iou': tensor(0.5833), 'Loss': 0.07223287224769592}
{'iou_aguada': 1.0, 'iou_building': 0.25, 'iou_platform': 0.5}

val_avg_iou: 0.6007575392723083, val_loss: 0.07247784442522309
{'val_iou_aguada': 0.9636363387107849, 'val_iou_building': 0.36818182468414307, 'val_iou_platform': 0.47045454382896423}
EPOCH: 26


{'avg_iou': tensor(0.6667), 'Loss': 0.11303803324699402}
{'iou_aguada': 1.0, 'iou_building': 0.5, 'iou_platform': 0.5}
{'avg_iou': tensor(0.6667), 'Loss': 0.06175689771771431}
{'iou_aguada': 1.0, 'iou_building': 0.25, 'iou_platform': 0.75}
{'avg_iou': tensor(0.7500), 'Loss': 0.06912446022033691}
{'iou_aguada': 1.0, 'iou_building': 0.5, 'iou_platform': 0.75}
{'avg_iou': tensor(0.4167), 'Loss': 0.08984110504388809}
{'iou_aguada': 1.0, 'iou_building': 2.5014246674004426e-08, 'iou_platform': 0.25}
{'avg_iou': tensor(0.5000), 'Loss': 0.0644170269370079}
{'iou_aguada': 1.0, 'iou_building': 4.508170572958825e-11, 'iou_platform': 0.5}
{'avg_iou': tensor(0.5000), 'Loss': 0.07863223552703857}
{'iou_aguada': 1.0, 'iou_building': 6.251169026860737e-11, 'iou_platform': 0.5}

val_avg_iou: 0.6007575392723083, val_loss: 0.07009002945639871
{'val_iou_aguada': 0.9636363387107849, 'val_iou_building': 0.36818182468414307, 'val_iou_platform': 0.47045454382896423}
EPOCH: 27


{'avg_iou': tensor(0.6667), 'Loss': 0.05898946151137352}
{'iou_aguada': 1.0, 'iou_building': 0.5, 'iou_platform': 0.5}
{'avg_iou': tensor(0.3333), 'Loss': 0.09213809669017792}
{'iou_aguada': 1.0, 'iou_building': 2.4302938134157515e-11, 'iou_platform': 2.453711886452048e-11}
{'avg_iou': tensor(0.5000), 'Loss': 0.11438547819852829}
{'iou_aguada': 1.0, 'iou_building': 0.25, 'iou_platform': 0.25}
{'avg_iou': tensor(0.9167), 'Loss': 0.05004718527197838}
{'iou_aguada': 1.0, 'iou_building': 1.0, 'iou_platform': 0.75}
{'avg_iou': tensor(0.7500), 'Loss': 0.052749887108802795}
{'iou_aguada': 1.0, 'iou_building': 0.5, 'iou_platform': 0.75}
{'avg_iou': tensor(0.6667), 'Loss': 0.0749727264046669}
{'iou_aguada': 1.0, 'iou_building': 0.5, 'iou_platform': 0.5}

val_avg_iou: 0.6007575392723083, val_loss: 0.07307910658419133
{'val_iou_aguada': 0.9636363387107849, 'val_iou_building': 0.36818182468414307, 'val_iou_platform': 0.47045454382896423}



##Предсказание, вывод файлов

###Пишем логи в txt

In [21]:
#print(log_loss)
#print(log_iou_class)
log_loss_txt = open('/content/drive/MyDrive/data/model/log_loss.txt', 'w') # Открываем файл
for i in range(len(log_loss)):
  log_loss_txt.write(str(log_loss[i]) + '\n')
log_loss_txt.close




<function TextIOWrapper.close>

In [22]:
log_iou_txt = open('/content/drive/MyDrive/data/model/log_iou_class.txt', 'w') # Открываем файл
for i in range(len(log_iou_class)):
  log_iou_txt.write(str(log_iou_class[i]) + '\n')
log_iou_txt.close

<function TextIOWrapper.close>

###Сохраняем модель

In [24]:
filepath = '/content/drive/MyDrive/data/model_dls.pth'
torch.save(model.state_dict(), filepath)



In [25]:
#Later to restore:
model.load_state_dict(torch.load(filepath))
model.eval()

DeepLabV3(
  (model): DeepLabV3(
    (backbone): IntermediateLayerGetter(
      (conv1): Conv2d(13, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          

###Делаем предсказание тестовой части данных

In [16]:
from scipy import sparse
from PIL import Image, ImageOps
import os
from datetime import datetime

#!mkdir predictions

In [17]:
model.eval()
# Мы будем итерироваться по одному  элементу из тестового датасета
count_operation = 0
for (X, _), tile_id in zip(test_ds, test_ds.ids):
    with torch.no_grad():
        X = X.unsqueeze(0).to(device)
        pred = model(X)
        masks = discretize_segmentation_maps(pred, config.thresh).squeeze(0)
        masks = masks.cpu().numpy()
        #masks = 255 - (masks * 255).astype(np.byte)

        for i, class_name in enumerate(test_ds.classes):
            img = Image.fromarray((masks[i] * 255).astype(np.uint8))
            img = ImageOps.invert(img)
            img.save(f'/content/drive/MyDrive/data/model/predict_tif/tile_{tile_id}_mask_{class_name}.tif')
            count_operation += 1
            
print(f'Сохранено {count_operation} изображений. Операция завершена {datetime.now()}')#Контроль объема исходящих данных   

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:387: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Сохранено 987 изображений. Операция завершена 2021-06-28 08:43:25.887151


In [18]:
#!rm -R predictions_tiff
#!mkdir predictions_tiff
def convert_image(img_path):
    img=Image.open(img_path)
    return sparse.csr_matrix(ImageOps.invert(img),dtype=bool)
count_operation = 0
for file in os.listdir('/content/drive/MyDrive/data/model/predict_tif/'):
    fname = os.path.join('/content/drive/MyDrive/data/model/pred/', file)
    fname_tif = os.path.join('/content/drive/MyDrive/data/model/predict_tif/', file)
    sparse.save_npz(os.path.splitext(fname)[0]+'.npz', convert_image(fname_tif), compressed=True)
    count_operation += 1
print(f'Сконвертировано {count_operation} изображений. Операция завершена {datetime.now()}')#Контроль объема исходящих данных

#clear_output()

Сконвертировано 987 изображений. Операция завершена 2021-06-28 08:43:37.149460


In [19]:

!zip ./my_preds_G2806_27epoch.zip /content/drive/MyDrive/data/model/pred/*

  adding: content/drive/MyDrive/data/model/pred/tile_1765_mask_aguada.npz (deflated 62%)
  adding: content/drive/MyDrive/data/model/pred/tile_1765_mask_building.npz (deflated 62%)
  adding: content/drive/MyDrive/data/model/pred/tile_1765_mask_platform.npz (deflated 62%)
  adding: content/drive/MyDrive/data/model/pred/tile_1766_mask_aguada.npz (deflated 62%)
  adding: content/drive/MyDrive/data/model/pred/tile_1766_mask_building.npz (deflated 62%)
  adding: content/drive/MyDrive/data/model/pred/tile_1766_mask_platform.npz (deflated 62%)
  adding: content/drive/MyDrive/data/model/pred/tile_1767_mask_aguada.npz (deflated 62%)
  adding: content/drive/MyDrive/data/model/pred/tile_1767_mask_building.npz (deflated 62%)
  adding: content/drive/MyDrive/data/model/pred/tile_1767_mask_platform.npz (deflated 62%)
  adding: content/drive/MyDrive/data/model/pred/tile_1768_mask_aguada.npz (deflated 62%)
  adding: content/drive/MyDrive/data/model/pred/tile_1768_mask_building.npz (deflated 62%)
  addin

##Wandb Отчет

https://wandb.ai/matroskin_sk/uncategorized/reports/Untitled-Report--Vmlldzo4MDg0MDU

In [27]:
data_csv = pd.read_csv('/content/drive/MyDrive/data/wandb_export_2021-06-28T09_21_55.795+03_00.csv')
data_csv

,Name,State,Notes,User,Tags,Created,Runtime,Sweep,batch_size,crop_size,epochs,learning_rate,lr,momentum,num_classes,pretrained,resize_max,resize_min,sentinel1_bands,sentinel2_bands,threshold,avg_iou,iou_aguada,iou_building,iou_platform,loss,val_avg_iou,val_iou_aguada,val_iou_building,val_iou_platform,val_loss
0,crisp-surf-15,crashed,-,matroskin_sk,NaN,2021-06-17T07:13:23.000Z,1978,NaN,8.0,480.0,15.0,NaN,0.0001,0.9,3.0,True,480.0,480.0,[],[],NaN,0.048180,5.000425e-12,4.756230e-02,9.697625e-02,0.655881,NaN,NaN,NaN,NaN,NaN
1,polar-resonance-14,crashed,-,matroskin_sk,NaN,2021-06-17T05:45:35.000Z,2763,NaN,4.0,480.0,30.0,NaN,0.0010,0.9,3.0,True,480.0,480.0,[],[],NaN,0.101041,1.489149e-02,3.265309e-03,2.849658e-01,0.692901,NaN,NaN,NaN,NaN,NaN
2,happy-music-13,crashed,-,matroskin_sk,NaN,2021-06-16T19:10:44.000Z,34669,NaN,4.0,480.0,30.0,NaN,0.0010,0.9,3.0,True,480.0,480.0,[],[],NaN,0.911320,1.000000e+00,8.361372e-01,8.978235e-01,0.006393,0.778344,0.980265,0.682946,0.671821,0.017510
3,avid-violet-12,crashed,-,matroskin_sk,NaN,2021-06-16T16:00:25.000Z,11295,NaN,4.0,480.0,30.0,NaN,0.0010,0.9,3.0,True,480.0,480.0,[],[],NaN,0.800463,1.000000e+00,6.696801e-01,7.317081e-01,0.019510,0.732898,0.961712,0.653972,0.583010,0.027815
4,playful-star-11,finished,-,matroskin_sk,NaN,2021-06-16T13:46:40.000Z,7875,NaN,4.0,480.0,10.0,NaN,0.0010,0.9,3.0,True,480.0,480.0,[],[],NaN,0.444444,1.000000e+00,1.388362e-11,3.333333e-01,0.073386,0.581832,0.954955,0.340090,0.450450,0.066606
5,fluent-cloud-10,finished,-,matroskin_sk,NaN,2021-06-16T08:54:10.000Z,17466,NaN,4.0,480.0,10.0,NaN,0.0010,0.9,3.0,True,480.0,480.0,[],[],NaN,0.007154,3.621674e-11,2.139492e-02,6.751130e-05,0.018235,0.052050,0.001051,0.081735,0.073364,0.040129
6,laced-elevator-9,crashed,-,matroskin_sk,NaN,2021-06-16T08:30:15.000Z,1373,NaN,4.0,480.0,10.0,NaN,0.0010,0.9,3.0,True,480.0,480.0,[],[],NaN,0.833333,1.000000e+00,7.500000e-01,7.500000e-01,0.288087,NaN,NaN,NaN,NaN,NaN
7,ethereal-sponge-8,crashed,-,matroskin_sk,NaN,2021-06-16T07:29:03.000Z,3564,NaN,4.0,480.0,10.0,NaN,0.0010,0.9,3.0,True,480.0,480.0,[],[],NaN,0.333333,1.000000e+00,2.060672e-10,2.845415e-11,0.249430,NaN,NaN,NaN,NaN,NaN
8,comic-pyramid-7,crashed,-,matroskin_sk,NaN,2021-06-16T07:18:13.000Z,11260,NaN,4.0,480.0,10.0,NaN,0.0010,0.9,3.0,True,480.0,480.0,[],[],NaN,0.025363,2.692981e-12,6.431034e-02,1.177943e-02,0.659412,NaN,NaN,NaN,NaN,NaN
9,usual-shadow-6,finished,-,matroskin_sk,NaN,2021-06-15T18:35:49.000Z,435,NaN,4.0,480.0,20.0,NaN,0.0010,0.9,3.0,True,480.0,480.0,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
